# Data Engineering 1: Graded Lab 02
---------------

#### Grading
For this graded lab you can get a total of 20 points. These 20 points count 10% of your final grade for the course.

#### Start
Start of the Graded Lab 02 is **Wednesday, May 04th at 23:55**.

#### Deadline
Deadline for the submission of the Graded Lab 02 is **Wednesday, May 18th at 23:59**.

#### Note
Check each result carefully. Use data filter, cleaning, and transformation methods wherever needed. The data can sometimes be really messy and have hidden issues.

#### Submission
You are allowed to submit the solution in groups of **two or three** students.
Submit your GradedLab02.ipynb file renamed to FirstnameStudent01LastnameStudent01_FirstnameStudent02LastnameStudent02_FirstnameStudent03LastnameStudent03.ipynb in moodle.   
Please submit a runnable python jupyter notebook file.
All other submissions will be rejected and graded with 0 points.

#### Task 01: Data Engineering with MongoDB [8 points].    
The 'restaurants_mongodb.json' file contains a dataset with a collection of restaurant information. In this task we work with this data and the MongoDB database. Run a MongoDB community server (see here https://www.mongodb.com/try/download/community) on your machine and connect to it with the python library 'pymongo'. It is also allowed to formulate and run the queries by using a MongoDB GUI. If you choose this option just copy and paste the queries into the cell (c) below.

##### __(a)  Create a database called data and a collection called restaurants in the database data.__ 

In [ ]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["data"]

col = db["restaurants"]

db.restaurants.drop()

##### __(b)  Insert the documents from the file 'restaurants.json' into the collection restaurants.__ 

In [ ]:
import json

json_file = 'restaurants_mongodb.json'
data = [json.loads(line) for line in open(json_file, 'r')]

col.insert_many(data)

# show the first inserted document
print(col.find_one())

##### (c)  Write the queries to print the requested results.

* (1) write the query to print the total number of documents in the collection 'restaurants'
* (2) write the query to print 5 restaurant names with 'Bronx' as borough from the collection 'restaurants'
* (3) Write the query to print the restaurant names that achieved a score, more than 80 but less than 100 from the c collection 'restaurants'
* (4) Write the query to print the restaurant names for those restaurants which contain 'Wil' as first three letters in its name.
* (5) Write the query to print the restaurant names which are not belonging to the borough Staten Island or Queens or Bronx or Brooklyn.
* (6) Write the query to print each borough and the number of restaurants for each borough (hint: use the grouping function)
* (7) Write the query to print the restaurant names which have the term 'Mexican' in the name attribute but don't have mexican as content in the cuisine attribute.
* (8) Write the query to print all names of bakeries with 'sugar' in the name.

In [ ]:
# (1) Find the total number of documents in the collection 'restaurants'.
print("\n(1)")
print(col.count_documents({}))

# (2) Print 5 restaurant names with 'Bronx' as the borough.
print("\n(2)")
cur = col.find({"borough": "Bronx"}).limit(5)
for doc in cur:
    if doc["name"]:
        print(doc["name"])
    else:
        print(f"No name but id is {doc['_id']}")

# (3) Print the restaurant names that achieved a score, more than 80 but less than 100.
print("\n(3)")
cur = col.find({"grades.score": {"$gt": 80, "$lt": 100}})
for doc in cur:
    print(doc["name"])


In [ ]:
# (4) Print the restaurant names that contain 'Wil' as the first three letters in its name.
print("\n(4)")
cur = col.find({"name": {"$regex": "^Wil"}})
for doc in cur:
    print(doc["name"])
    

In [ ]:
# (5) Write the query to print the restaurant names which are not belonging to the borough Staten Island or Queens or Bronx or Brooklyn.
print("\n(5)")
cur = col.find({"borough": {"$nin": ["Staten Island", "Queens", "Bronx", "Brooklyn"]}})
for doc in cur:
    print(doc["name"])

In [ ]:
# (6) Print each borough and the number of restaurants for each borough.
print("\n(6)")
cur = col.aggregate([
    {"$group": {"_id": "$borough", "count": {"$sum": 1}}}
])
for doc in cur:
    print(doc["_id"], doc["count"])


In [ ]:
# Print the restaurant names which have the term 'Mexican' in the name attribute but don't have mexican as content in the cuisine attribute.
print("\n(7)")
cur = col.find({"name": {"$regex": "Mexican"}, "cuisine": {"$ne": "Mexican"}})
for doc in cur:
    print(doc["name"])


In [ ]:
# Print all names of bakeries with 'sugar' in the name.
print("\n(8)")
cur = col.find({"name": {"$regex": "sugar", "$options" : "i"}, "cuisine": {"$regex": "(?i)bakery"}})
for doc in cur:
    print(doc["name"])

#### Task 02:  Data Engineering with  BaseX [8 points]. 
Install the BaseX XML database and create a database with the file 'restaurants_basex.json'. Formulate all queries from the Task 01 (1) to (8) with XPath/XQuery in BaseX. The content of both json files is the same, so you should retrieve the same results as with the MongoDB solution. You can execute the queries directly in the BaseX GUI. After succesful execution in BaseX just copy and paste the queries into the cell below. If a query is not expressible in XPath/XQuery write down not possible and the reason.

Answer...

(1) 
count(collection("restaurants")//address)

(2)
(for $item in collection("restaurants")//_
where $item/borough = "Bronx"
return $item/name/text())[position() = 1 to 5]

(3) 
???

(4)
for $item in collection("restaurants")//_
where matches($item/name, "^Wil")
return $item/name/text()

(5)
for $item in collection("restaurants")//_
where not(matches($item/borough, "Staten Island"))
and not(matches($item/borough, "Queens"))
and not(matches($item/borough, "Bronx"))
and not(matches($item/borough, "Brooklyn"))
return $item/name/text()

(6)
for $item in collection("restaurants")//_
let $item_ref := $item/borough
group by $item_ref
return($item_ref, count($item))

(7)
for $item in collection("restaurants")//_
where matches($item/name, "Mexican")
and not(matches($item/cuisine, "Mexican"))
return $item/name/text()

(8)
for $item in collection("restaurants")//_
where matches($item/cuisine, "Bakery")
and matches($item/name, "sugar", "i")
return $item/name/text()

#### Task 03: Conclusions [4 points]. 
Write 5-10 sentences with your conclusions concerning performance, usability etc. between using MongoDB and BaseX.

When setting up BaseX, it quickly becomes apparent that the installation is much more complex and less self-explanatory than that of MongoDB. Since MongoDB is one of the most widely used document stores, it is easy to find relevant resources on the Internet if you have initial difficulties, which is less the case with BaseX. BaseX offers as well as MongoDB an interactive GUI, which is however also less user-friendly than that of MongoDB. MongoDB offers many server-side integrations, including Python with the library pymongo, with a very simple API that makes it very easy to create and manage a MongoDB database and write queries directly in Python, which is not the case with BaseX.
XQuery is a suitable tool to analyze XML data, which MongoDB is not.

MongoDB also offers the possibility to keep some or all structures only in main memory by means of in-memory support, which BaseX does not offer and thus represents a performance advantage.

